In [1]:
# Load Spark engine
!pip3 install pyspark
import findspark
findspark.init()

In [2]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

from pyspark.sql import functions as F
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
spark = SparkSession.builder.appName("StructuredStreaming_WordCount").getOrCreate()

In [4]:
# Create DataFrame representing the stream of input lines
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 7000).load()

In [5]:
# Split the lines into words
words = lines.select(explode(split(lines.value, " ")).alias("word"))
wordcounts=words.groupBy("word").count()

In [6]:
preprocessed_word = words.select(
          "word", 
          regexp_replace('word', '[^a-zA-Z]+', '').alias('preprocessed_word')
           )
   
   
preprocessed_word = preprocessed_word.dropDuplicates()
preprocessed_word = preprocessed_word.withColumn("Word_Length",length(col("preprocessed_Word")))
preprocessed_word = preprocessed_word.withColumn("Length_Class", when((col("Word_Length") < 2), "null")
                                       .when((col("Word_Length") >= 6), 6)
                                       .otherwise(col("Word_Length"))
                                      )
new_word = preprocessed_word.groupBy("Length_Class").count().where(col("Length_Class") >= 2).orderBy('Length_Class', ascending=True)

In [7]:
query = new_word.writeStream.outputMode("complete").format("console").start()

# streamed input in the 7000 port
#I am a student, my name is mousumi, hello mou, hello 123, how are you
#I am fine, is it somethng interesting with you, can you tell me that?




In [ ]:
query.awaitTermination()